# Country Searcher

In [2]:
import json
import pandas as pd
import pydeck as pdk
from geopy.geocoders import Nominatim
import geopandas as gpd
from shapely.geometry import Point
import random
from pymongo import MongoClient

# Replace with your API key from Elsevier
API_KEY = "9dd590e8ec8a8513452bd2912d8b0e9d"
BASE_URL = 'https://api.elsevier.com/content/search/scopus'


In [3]:
def loadJsonData(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            data = json.load(file)
    except UnicodeDecodeError:
        try:
            with open(filename, 'r', encoding='latin-1') as file:
                data = json.load(file)
        except UnicodeDecodeError:
            return "Failed to load data. Please check the file encoding."
    except FileNotFoundError:
        return "Failed to load data. File not found."
    return data

# Load data
data = loadJsonData(r'E:\Works\Intro-to-Data-Science\2110403_DSDE_Final_Project\src\data\db-data\predicted_data.json')
df = pd.DataFrame(data)

In [4]:
from geopy.geocoders import Nominatim

def locate_place(address):
    geolocator = Nominatim(user_agent="geoapi", timeout=10)
    location = geolocator.geocode(address, language="en")

    if location:
        # Extract the last part of the address (country name)
        localized_country_name = location.address.split(",")[-1].strip()
        # standardized_country_name = get_country_name_in_english(localized_country_name)
        split_address = location.address.split(",")
        if len(split_address) > 1:
            country_name = split_address[-1].strip()
        else:
            country_name = location.address
        # print(country_name)
        # print(localized_country_name)
        # print(standardized_country_name)
        return country_name
    else:
        return "None"
        


In [ ]:
count = df.groupby('affiliation').size().reset_index(name='count')
count 
affiliation_list = count['affiliation'].tolist()

affiliation_list = set(affiliation_list)

_dict = {}
count = 0

for affiliation in affiliation_list:
    split_data = affiliation.split(',')
    split_data = [a.strip() for a in split_data]
    if len(split_data) > 1:
        location = split_data[-1]
    else:
        location = affiliation.strip()
    
    country_name = locate_place(location)
    _dict.update({affiliation: country_name})
    # print(affiliation, country_name)
    # count += 1
    # if count == 100:
    #     break


In [ ]:
for key, val in _dict.items():
    if val == None:
        val = "None"
    print(key, val)

In [ ]:
with open(r'E:\Works\Intro-to-Data-Science\2110403_DSDE_Final_Project\src\data\db-data\affiliation_country.json', 'w') as f:
    json.dump(_dict, f, indent=4)

In [ ]:
def mapper(x):
    if x == "Unknown":
        return None
    
    if x in _dict:
        return _dict[x]
    else:
        return None

In [ ]:
df = pd.DataFrame(data)
df['country_name'] = df['affiliation'].apply(lambda x: mapper(x))
df

In [ ]:
df.to_json(r'E:\Works\Intro-to-Data-Science\2110403_DSDE_Final_Project\src\data\db-data\predicted_data_with_country_name.json', orient='records', lines=True)

# Latitude Longitude from country name

In [8]:
from geopy.geocoders import Nominatim

def get_lat_lon(country_name):
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.geocode(country_name)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# Example usage



In [23]:
data = loadJsonData(r'E:\Works\Intro-to-Data-Science\2110403_DSDE_Final_Project\src\data\db-data\predicted_data_with_country.json')
df = pd.DataFrame(data)
country = df['country_name'].tolist()
country = set(country)
len(country)


164

In [24]:
lat_long_dict = {}
count = 0
for c in country:
    lat, lon = get_lat_lon(c)
    print(count, c, lat, lon)
    lat_long_dict.update({c: (lat, lon)})
    count += 1
    
for key, val in lat_long_dict.items():
    print(key, val[0], val[1])

0 Russia 64.6863136 97.7453061
1 Saudi Arabia 25.6242618 42.3528328
2 Lithuania 55.3500003 23.7499997
3 Czechia 49.7439047 15.3381061
4 France 46.603354 1.8883335
5 South Korea 36.638392 127.6961188
6 Norway 64.5731537 11.52803643954819
7 Jordan 44.6663146 -93.6261918
8 Singapore 1.357107 103.8194992
9 Chad 15.6134137 19.0156172
10 None 44.933143 7.540121
11 Costa Rica 10.2735633 -84.0739102
12 Poland 52.215933 19.134422
13 Ireland 52.865196 -7.9794599
14 Barbados 13.1500331 -59.5250305
15 Hungary 47.1817585 19.5060937
16 Latvia 56.8406494 24.7537645
17 Netherlands 52.2434979 5.6343227
18 Fiji -18.1239696 179.0122737
19 Kuwait 29.3796532 47.9734174
20 Vietnam 15.9266657 107.9650855
21 Iraq 33.0955793 44.1749775
22 Federated States of Micronesia 8.6062347 151.832744331612
23 Iceland 64.9841821 -18.1059013
24 Bhutan 27.549511 90.5119273
25 Pakistan 30.3308401 71.247499
26 Lebanon 40.375713 -76.4626118
27 Burundi -3.426449 29.9324519
28 Ethiopia 10.2116702 38.6521203
29 Togo 8.7800265 1.0

In [25]:
with open(r'E:\Works\Intro-to-Data-Science\2110403_DSDE_Final_Project\src\data\db-data\country_lat_lon.json', 'w') as f:
    json.dump(lat_long_dict, f, indent=4)

In [37]:
def lat_mapper(x):
    if x == None:
        return None
    if x in lat_long_dict:
        return lat_long_dict[x][0]
    else:
        return None
    
def lon_mapper(x):
    if x == None:
        return None
    if x in lat_long_dict:
        return lat_long_dict[x][1]
    else:
        return None

In [38]:
df['latitude'] = df['country_name'].apply(lambda x: lat_mapper(x))
df['longitude'] = df['country_name'].apply(lambda x: lon_mapper(x))
print(lat_mapper('Thailand'), lon_mapper('Thailand'))
df

14.8971921 100.83273


,title,name,affiliation,subject_code,cited_count,pub_type,year,predict_gender,country_name,latitude,longitude
0,Public health and international epidemiology f...,Krit,Thailand,2700,1,Book,2018,m,Thailand,14.897192,100.832730
1,Public health and international epidemiology f...,Krit,United States,2700,1,Book,2018,m,United States,39.783730,-100.445882
2,Flexible Printed Active Antenna for Digital Te...,Teerapong,Thailand,2208,1,Conference Proceeding,2018,m,Thailand,14.897192,100.832730
3,Flexible Printed Active Antenna for Digital Te...,Panuwat,Thailand,2208,1,Conference Proceeding,2018,m,Thailand,14.897192,100.832730
4,Flexible Printed Active Antenna for Digital Te...,Teerapong,Thailand,2504,1,Conference Proceeding,2018,m,Thailand,14.897192,100.832730
...,...,...,...,...,...,...,...,...,...,...,...
300359,Biotransformation of 8:2 polyfluoroalkyl phosp...,Noelia,Unknown,2305,24,Journal,2017,f,None,NaN,NaN
300360,Biotransformation of 8:2 polyfluoroalkyl phosp...,Adrian,Unknown,2305,24,Journal,2017,m,None,NaN,NaN
300361,Biotransformation of 8:2 polyfluoroalkyl phosp...,Jonathan,Unknown,2305,24,Journal,2017,m,None,NaN,NaN
300362,Biotransformation of 8:2 polyfluoroalkyl phosp...,Ailette,Unknown,2305,24,Journal,2017,f,None,NaN,NaN


In [39]:
df.to_json(r'E:\Works\Intro-to-Data-Science\2110403_DSDE_Final_Project\src\data\db-data\predicted_data_with_country_lat_lon.json', orient='records', lines=True)